In [39]:
from tickers import Ticker, scrape_tickers, query_list, recent_IPO_list, upcoming_IPO_list

# Scrape to get updated tickers and company names for stocks and ETFs
scrape_tickers()

import_lists = [query_list, recent_IPO_list, upcoming_IPO_list]

tickers = []

# Build raw query list with ticker abbrevation and name of company
for i in import_lists:
    for item in i:
        tickers.append(item)

In [4]:
import pandas as pd

submissions_df = pd.read_csv('submissions.csv')
comments_df = pd.read_csv('comments.csv')

In [5]:
submissions_df['Body_Words'] = submissions_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')
submissions_df['Title_Words'] = submissions_df.Title.str.replace("'", "").str.strip().str.split('[\W_]+')

In [40]:
common_words = ['Corp', 'Corporation', 'Ltd', 'Acquisition', 'Pharmaceuticals', 'Holding', 'Group']
new_name = ""
ticker_names = []
for ticker in tickers:
    new_name = ""
    company_words = ticker.name.split()
    for word in company_words:
        if word in common_words:
            continue
        else:
            if new_name == "":
                new_name = word
            else:
                new_name = new_name + " " + word
    ticker.name = new_name

In [41]:
for ticker in tickers:
    print(ticker.abbrev)
    print(ticker.name)

A
Agilent Technologies
AA
Alcoa
AAC
Ares
AACE
Alexandria Agtech/Climate Innovation
AACG
ATA Creativity Global
AACO
Advancit I
AACP
Aeon
AACQ
Artius
AAIC
Arlington Asset Investment
AAL
American Airlines
AAMC
Altisource Asset Management
AAME
Atlantic American
AAN
The Aaron's Company
AAOI
Applied Optoelectronics
AAON
AAON, Inc.
AAP
Advance Auto Parts
AAPL
Apple
AAQC
Accelerate
AAT
American Assets Trust
AAU
Almaden Minerals
AAWW
Atlas Air Worldwide Holdings
AB
AllianceBernstein
ABB
ABB Ltd.
ABBV
AbbVie
ABC
AmerisourceBergen
ABCB
Ameris Bancorp
ABCFF
Abacus Mining & Exploration
ABCL
AbCellera Biologics
ABCM
Abcam PLC
ABCO
Able Brands
ABCP
AmBase
ABEO
Abeona Therapeutics
ABEV
Ambev
ABG
Asbury Automotive
ABGI
ABG I
ABIO
ARCA biopharma
ABM
ABM Industries
ABMC
American Bio Medica
ABMD
Abiomed
ABNB
Airbnb
ABR
Arbor Realty Trust
ABST
Absolute Software
ABT
Abbott Laboratories
ABTX
Allegiance Bancshares
ABUS
Arbutus Biopharma
AC
Associated Capital
ACA
Arcosa
ACAC
Acies
ACAD
ACADIA
ACAH
Atlantic Coa

BSM
Black Stone Minerals
BSMX
Banco Santander Mexico
BSN
Broadstone
BSPE
Bespoke Capital
BSQR
Bsquare
BSRR
Sierra Bancorp
BSTG
Biostage
BSVN
Bank7 Corp.
BSX
Boston Scientific
BSY
Bentley Systems
BTAI
BioXcel Therapeutics
BTAQ
Burgundy Technology
BTBT
Bit Digital
BTCM
BIT Mining
BTCY
Biotricity
BTFL
Bountiful Co.
BTG
B2Gold
BTHE
Boston Therapeutics
BTI
British American Tobacco
BTN
Ballantyne Strong
BTNB
Bridgetown 2 Holdings
BTNC
Bridgetown 3 Holdings
BTRS
BTRS Holdings
BTU
Peabody Energy
BTVC
Tribe Capital Growth II
BTWN
Bridgetown Holdings
BTX
Brooklyn ImmunoTherapeutics
BTZI
BOTS, Inc.
BUD
Anheuser-Busch InBev
BUR
Burford Capital
BURL
Burlington Stores
BUSE
First Busey
BV
BrightView Holdings
BVH
Bluegreen Vacations
BVN
Compania de Minas Buenaventura
BVS
Bioventus
BVXV
BiondVax
BW
Babcock & Wilcox Enterprises
BWA
BorgWarner
BWAC
Better World
BWAY
Brainsway
BWB
Bridgewater Bancshares
BWEN
Broadwind Energy
BWFG
Bankwell Financial
BWL.A
Bowl America
BWMN
Bowman Consulting
BWMX
Betterware

Discover Financial Services
DG
Dollar General
DGICA
Donegal
DGICB
Donegal
DGII
Digi International
DGLY
Digital Ally
DGNR
Dragoneer Growth Opportunities
DGNS
Dragoneer Growth Opportunities II
DGNU
Dragoneer Growth Opportunities III
DGON
Black Dragon I
DGX
Quest Diagnostics
DHBC
DHB Capital
DHC
Diversified Healthcare Trust
DHCA
DHC
DHCC
Diamondhead Casino
DHHC
DiamondHead Holdings
DHI
D.R. Horton
DHIL
Diamond Hill Investment
DHR
Danaher
DHT
DHT Holdings
DHX
DHI
DIBS
1stdibs.com
DIIBF
Dorel Industries
DIII
DD3 III
DILA
DILA Capital
DIN
Dine Brands Global
DIOD
Diodes Incorporated
DIS
Disney
DISA
Disruptive I
DISCA
Discovery
DISCB
Discovery
DISCK
Discovery
DISH
DISH Network
DIST
Distoken
DIT
AMCON Distributing
DJCO
Daily Journal
DK
Delek US Holdings
DKDCA
Data Knights
DKL
Delek Logistics Partners
DKNG
DraftKings
DKS
Dick's Sporting Goods
DL
China Distance Education
DLA
Delta Apparel
DLAKY
Deutsche Lufthansa AG
DLB
Dolby Laboratories
DLCA
Deep Lake Capital
DLGNF
Dialog Semiconductor
DLHC
DLH

Genetic Technologies
GENH
Generation Hemp
GENI
Genius Sports
GENN
Genesis Healthcare
GEO
GEO
GEOS
Geospace Technologies
GERN
Geron
GES
Guess
GEVO
Gevo, Inc.
GFASY
Gafisa
GFED
Guaranty Federal Bancshares
GFF
Griffon
GFI
Gold Fields
GFL
GFL Environmental
GFLD
Gefen Landa
GFN
General Finance
GFOR
Graf IV
GFX
Golden Falcon
GGAL
Grupo Financiero Galicia
GGB
Gerdau
GGG
Graco
GGGV
G3 VRM
GGI
Guggenheim Special Purpose I
GGMC
Glenfarne Merger
GGPI
Gores Guggenheim
GH
Guardant Health
GHAC
Gaming & Hospitality
GHC
Graham Holdings
GHG
GreenTree Hospitality
GHL
Greenhill & Co.
GHLD
Guild Holdings Company
GHM
Graham
GHSI
Guardion Health Sciences
GHVI
Gores Holdings VI
GIA
GigCapital5
GIB
CGI, Inc
GIF
GigCapital6
GIFI
Gulf Island Fabrication
GIG
GigCapital4
GIGA
Giga-tronics
GIGM
GigaMedia
GIII
G-III Apparel
GIIX
Gores Holdings VIII
GIL
Gildan Activewear
GILD
Gilead Sciences
GILT
Gilat Satellite Networks
GIS
General Mills
GIW
GigInternational1
GIX
GigCapital2
GKOS
Glaukos
GL
Globe Life
GLAD
Gladston

JCTCF
Jewett-Cameron Trading
JD
JD.com
JEF
Jefferies Financial
JELD
JELD-WEN
JENGQ
Just Energy
JFIN
Jiayin
JFU
9F Inc.
JG
Aurora Mobile
JHG
Janus Henderson
JHX
James Hardie Industries
JIH
Juniper Industrial Holdings
JILL
J.Jill, Inc.
JJSF
J&J Snack Foods
JKHY
Jack Henry & Associates
JKS
JinkoSolar
JLL
Jones Lang LaSalle
JMIA
Jumia Technologies AG
JMP
JMP
JNCE
Jounce Therapeutics
JNJ
Johnson & Johnson
JNPR
Juniper Networks
JOAN
JOANN
JOB
GEE
JOBS
51job
JOE
The St. Joe Company
JOFF
JOFF Fintech
JOUT
Johnson Outdoors
JP
Jupai Holdings
JPM
JPMorgan Chase
JRJC
China Finance Online
JRSH
Jerash Holdings
JRVR
James River Holdings
JSDA
Jones Soda Co.
JT
Jianpu Technology
JUGG
Jaws Juggernaut
JUN
Juniper II
JUPW
Jupiter Wellness
JUVF
Juniata Valley Financial
JVA
Coffee Co.
JW.A
John Wiley & Sons
JW.B
John Wiley & Sons
JWEL
Jowell Global
JWN
Nordstrom
JWS
Jaws
JWSM
Jaws Mustang
JYAC
Jiya
JYNT
The Joint Corp.
JZXN
Jiuzi Holdings
K
Kellogg
KAHC
KKR Holdings I
KAI
Kadant
KAII
Kismet Two
KAIR
Kairos


...

VRNS
Varonis Systems
VRNT
Verint Systems
VRPX
Virpax
VRRM
Verra Mobility
VRS
Verso
VRSK
Verisk Analytics
VRSN
VeriSign
VRT
Vertiv Holdings
VRTS
Virtus Investment Partners
VRTV
Veritiv
VRTX
Vertex
VS
Versus Systems
VSAT
ViaSat
VSEC
VSE
VSH
Vishay Intertechnology
VST
Vistra Energy
VSTA
Vasta Platform
VSTM
Verastem
VSTO
Vista Outdoor
VTAQ
Ventoux CCM
VTGN
VistaGen Therapeutics
VTIQ
VectoIQ II
VTNR
Vertex Energy
VTOL
Bristow
VTR
Ventas
VTRS
Viatris
VTRU
Vitru
VTSI
VirTra
VTVT
vTv Therapeutics
VUZI
Vuzix
VVI
Viad
VVNT
Vivint Smart Home
VVOS
Vivos Therapeutics
VVPR
VivoPower International
VVV
Valvoline
VWAGY
Volkswagen AG
VWTR
Vidler Water Resources
VXRT
Vaxart
VYGG
Vy Global Growth
VYGR
Voyager Therapeutics
VYNE
VYNE Therapeutics
VYNT
Vyant Bio
VZ
Verizon
VZIO
VIZIO
W
Wayfair
WAB
Wabtec
WABC
Westamerica Bancorporation
WAFD
Washington Federal
WAFU
Wah Fu Education
WAL
Western Alliance Bancorporation
WALD
Waldencast
WARM
Cool Technologies
WARR
Warrior Technologies
WASH
Washington Trust Bancor

EUO
ProShares UltraShort Euro
EURL
Direxion Daily FTSE Europe Bull 3X Shares
EURZ
Xtrackers Eurozone Equity ETF
EUSA
iShares MSCI USA Equal Weighted ETF
EUSB
iShares ESG Advanced Total USD Bond Market ETF
EUSC
WisdomTree Europe Hedged SmallCap Equity Fund
EVX
VanEck Vectors Environmental Services ETF
EWA
iShares MSCI Australia ETF
EWC
iShares MSCI Canada ETF
EWCO
Invesco S&P 500 Equal Weight Communication Services ETF
EWD
iShares MSCI Sweden ETF
EWEB
Global X Emerging Markets Internet & E-commerce ETF
EWG
iShares MSCI Germany ETF
EWGS
iShares MSCI Germany Small-Cap ETF
EWH
iShares MSCI Hong Kong ETF
EWI
iShares MSCI Italy ETF
EWJ
iShares MSCI Japan ETF
EWJE
iShares MSCI Japan Equal Weighted ETF
EWJV
iShares MSCI Japan Value ETF
EWK
iShares MSCI Belgium ETF
EWL
iShares MSCI Switzerland ETF
EWM
iShares MSCI Malaysia ETF
EWMC
Invesco S&P MidCap 400 Equal Weight ETF
EWN
iShares MSCI Netherlands ETF
EWO
iShares MSCI Austria ETF
EWP
iShares MSCI Spain ETF
EWQ
iShares MSCI France ETF
EWRE
Inv

NORW
Global X MSCI Norway ETF
NTSX
WisdomTree 90/60 U.S. Balanced Fund
NUAG
NuShares Enhanced Yield U.S. Aggregate Bond ETF
NUBD
NuShares ESG U.S. Aggregate Bond ETF
NUDM
NuShares ESG International Developed Markets Equity ETF
NUEM
NuShares ESG Emerging Markets Equity ETF
NUGT
Direxion Daily Gold Miners Bull 2X Shares
NUHY
Nuveen ESG High Yield Corporate Bond ETF
NULC
Nuveen ESG Large-Cap ETF
NULG
NuShares ESG Large-Cap Growth ETF
NULV
NuShares ESG Large-Cap Value ETF
NUMG
NuShares ESG Mid-Cap Growth ETF
NUMV
NuShares ESG Mid-Cap Value ETF
NURE
NuShares Short-Term REIT ETF
NUSA
NuShares Enhanced Yield 1-5 Year U.S. Aggregate Bond ETF
NUSC
NuShares ESG Small-Cap ETF
NUSI
Nationwide Risk-Managed Income ETF
NVMZ
TrueShares Structured Outcome (November) ETF
NVQ
QRAFT AI-Enhanced U.S. Next Value ETF
NXTG
First Trust NASDAQ Smartphone Index Fund
NYF
iShares New York Muni Bond ETF
OBOR
KraneShares MSCI One Belt One Road ETF
OCIO
ClearShares OCIO ETF
OCTZ
TrueShares Structured Outcome (October

SPDR FactSet Innovative Technology ETF
XJH
iShares ESG Screened S&P Mid-Cap ETF
XJR
iShares ESG Screened S&P Small-Cap ETF
XLB
Materials Select Sector SPDR Fund
XLC
Communication Services Select Sector SPDR Fund
XLE
Energy Select Sector SPDR Fund
XLF
Financial Select Sector SPDR Fund
XLG
Invesco S&P 500 Top 50 ETF
XLI
Industrial Select Sector SPDR Fund
XLK
Technology Select Sector SPDR Fund
XLP
Consumer Staples Select Sector SPDR Fund
XLRE
Real Estate Select Sector SPDR Fund
XLSR
SPDR SSGA US Sector Rotation ETF
XLU
Utilities Select Sector SPDR Fund
XLV
Health Care Select Sector SPDR Fund
XLY
Consumer Discretionary Select Sector SPDR Fund
XME
SPDR S&P Metals & Mining ETF
XMHQ
Invesco S&P MidCap Quality ETF
XMLV
Invesco S&P MidCap Low Volatility ETF
XMMO
Invesco S&P MidCap Momentum ETF
XMPT
VanEck Vectors CEF Municipal Income ETF
XMVM
Invesco S&P MidCap Value with Momentum ETF
XNTK
SPDR NYSE Technology ETF
XOP
SPDR S&P Oil & Gas Exploration & Production ETF
XOUT
GraniteShares XOUT U.S. 

NCAC
Newcourt
RCPI
Reverence
ICGA
ICG Hypersonic
ACRO
Acropolis Infrastructure
FWAB
Fifth Wall II
CAVU
CAVU Technology
GWII
Good Works II
QPAA
Quiet Plus I
HCXX
Healthcare Merger II
DILA
DILA Capital
KETA
Keter1
CHWA
CHW
RCDA
Red Cell DRM
STLRA
Stellaris Growth
LRIS
Laris Media
AFAC
Arena Fortify
SHCA
Spindletop Health
HUMC
Anthropos Capital
TEGA
Think Elevation Capital Growth Opportunities
MITA
Coliseum
CBRG
Chain Bridge I
ONS
ONS
CLBR
Colombier
ETA
Integrated Energy Transition
LHIV
Lionheart IV
LHRT
Lionheart III
RGCB
Regencell Bioscience
SNTG
Sentage Holdings
ITAQ
Industrial Tech Acquisitions II
JACA
Jeneration
CATL
Category Leader Partner 1
ALTE
Altitude II
ALTT
Altitude III
LFTA
Lazard Fintech I
LHCA
Lazard Healthcare I
HTAQ
Hunt Companies I
JUGG
Jaws Juggernaut
IVCP
Swiftmerge
SDHI
Siddhi
WNNR
Andretti
GCSA
Gateway Strategic
AVEA
Avanea Energy
EOCO
Elliott Opportunity I
EOCW
Elliott Opportunity II
ALTP
Altamont Pharma
OSAA
OS
NHLD
NewHold Investment II
HZNA
Horizon III
HCAA
Hony 

In [7]:
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words
0,Submission,me6ezj,Drinking and Trading don’t mix. I accidentally...,CostantlyLost,Title says it all. Had a few too many drinks o...,NaN,NaN,208,t3_me6ezj,https://www.reddit.com/r/stocks/comments/me6ez...,1298,0.95,[1616818448.0],"[Title, says, it, all, Had, a, few, too, many,...","[Drinking, and, Trading, don, t, mix, I, accid..."
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,72,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida..."
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,81,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for..."
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,61,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about..."
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,97,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,..."


In [8]:
submissions_df["Tickers"] = ""
for index, row in submissions_df.iterrows():
    tickers_found = []
    body_list = row[12]
    title_list = row[13]
    body_set = set(body_list)
    title_set = set(title_list)
    for ticker in tickers:
        if ticker.abbrev == 'A' or ticker.abbrev == 'I':
            #if (ticker.name.split()[0] in body_set) or (ticker.name.split()[0] in title_set):
            if (ticker.name in body_set) or (ticker.name in title_set):
                tickers_found.append(ticker.abbrev)
        else:
            if (ticker.abbrev in body_set) or (ticker.abbrev in title_set):
                #print("Abbreviation match")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
            elif (ticker.name in body_set) or (ticker.name in title_set):
                #print("Name match")
                #print(f"{index}: {ticker.name}")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
    if len(tickers_found) == 0:
        submissions_df.at[index, "Tickers"] = ""
    else:
        submissions_df.at[index, "Tickers"]= tickers_found
        print(f"{index}: {tickers_found}")

1: ['ADS', 'BCS', 'DRI', 'HON', 'JP', 'MS', 'NDAQ', 'NIO', 'ORLY', 'REGN', 'RH', 'SGFY', 'TLS', 'U', 'DIA', 'QQQ', 'SPY']
2: ['DG', 'E', 'GOOGL', 'HD', 'SPGI', 'TMO', 'TXN', 'UNP', 'V']
3: ['PS']
4: ['AM', 'AMAT', 'BB', 'BNTX', 'CHWY', 'CIO', 'ENPH', 'ET', 'ETSY', 'FCX', 'FOR', 'GILD', 'KMX', 'KNDI', 'L', 'LB', 'LPL', 'LULU', 'MKC', 'MKC.V', 'MTD', 'MU', 'NDAQ', 'NEXT', 'NWG', 'OPEN', 'PM', 'SE', 'TREE', 'TSLA', 'TWOA', 'U', 'VIAC', 'VIACA', 'WBA']
6: ['FANG', 'PLTR', 'PS']
7: ['MX']
10: ['USD']
11: ['DD']
12: ['VIAC']
13: ['CEO', 'FRO', 'NAT', 'STNG', 'TK', 'TNK']
14: ['ARKK']
15: ['F', 'NIO', 'USD']
16: ['DD']
17: ['AMT', 'CCI', 'CTRE', 'EPRT', 'EQIX', 'INVH', 'NSA', 'PLD', 'PSB', 'REXR', 'REIT']
19: ['TSLA', 'VWAGY']
20: ['TWTR']
22: ['GME']
23: ['H', 'IJR', 'IWC', 'IWM', 'SDD']
28: ['CP']
29: ['TD']
30: ['NDAQ', 'TA', 'TWOA']
31: ['X', 'Y']
32: ['TMAC']
34: ['MSFT']
36: ['GME', 'TWOA', 'ARKK', 'LIT', 'MSOS', 'YOLO']
38: ['CEO', 'DHT', 'DSX', 'FRO', 'NAT', 'SFL', 'TNP', 'TWOA']
39: 

In [9]:
submissions_df = submissions_df.loc[submissions_df['Tickers'] != ""]
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,72,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ..."
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,81,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]"
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,61,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS]
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,97,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY..."
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,43,0.79,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]"


In [10]:
import nltk
#nltk.download('punkt')

In [11]:
# Using Vader Sentiment Analysis on Submissions
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pprint import pprint
#nltk.download('vader_lexicon')
sia = SIA()
results = []
submissions_df['negative_score'] = ""
submissions_df['neutral_score'] = ""
submissions_df['positive_score'] = ""
submissions_df['compound_score'] = ""
#submissions_loop = submissions_df[['Title', 'Body', 'Body_Words', 'Title_Words', 'Tickers']]
for index, row in submissions_df.iterrows():
    title = row[2]
    body = row[4]
    eval_text = title + " " + body
    body_words = row[12]
    title_words = row[13]
    tickers = row[14]
    pol_score= sia.polarity_scores(eval_text)
    #pol_score['submission_text'] = eval_text
    #pol_score['tickers'] = tickers
    #results.append(pol_score)
    submissions_df.at[index, 'negative_score'] = pol_score['neg']
    submissions_df.at[index, 'neutral_score'] = pol_score['neu']
    submissions_df.at[index, 'positive_score'] = pol_score['pos']
    submissions_df.at[index, 'compound_score'] = pol_score['compound']
        #submissions_loop['sentiment_scores'] = pol_score
#pprint(results[:3], width=100)

submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,72,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ...",0.045,0.848,0.107,0.9937
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,81,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]",0.042,0.836,0.122,0.9842
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,61,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS],0.142,0.778,0.081,-0.7559
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,97,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY...",0.031,0.871,0.098,0.9999
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,43,0.79,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]",0.066,0.746,0.188,0.9981


In [12]:
submissions_df['sentiment_label'] = 0
submissions_df.loc[submissions_df['compound_score'] > 0.2, 'sentiment_label'] = 1
submissions_df.loc[submissions_df['compound_score'] < -0.2, 'sentiment_label'] = -1
submissions_df

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,...,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,...,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ...",0.045,0.848,0.107,0.9937,1
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,...,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]",0.042,0.836,0.122,0.9842,1
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,...,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS],0.142,0.778,0.081,-0.7559,-1
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,...,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY...",0.031,0.871,0.098,0.9999,1
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,...,0.79,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]",0.066,0.746,0.188,0.9981,1
7,Submission,mdvvkm,Opportunity to gain about 10-14% on $MX (Bough...,RowanHarley,"Right now, $MX is trading far below what it wa...",Company Discussion,NaN,52,t3_mdvvkm,https://www.reddit.com/r/stocks/comments/mdvvk...,...,0.85,[1616785531.0],"[Right, now, MX, is, trading, far, below, what...","[Opportunity, to, gain, about, 10, 14, on, MX,...",[MX],0.085,0.763,0.152,0.8927,1
10,Submission,me0ast,Some figures summarizing what has been happeni...,futureIsYes,The Hang Seng Index contains all the Chinese g...,NaN,NaN,62,t3_me0ast,https://www.reddit.com/r/stocks/comments/me0as...,...,0.81,[1616798179.0],"[The, Hang, Seng, Index, contains, all, the, C...","[Some, figures, summarizing, what, has, been, ...",[USD],0.11,0.89,0,-0.8963,-1
11,Submission,mebzea,What stocks focus on sexual self-satisfaction?,Smur_,"To explain myself, I really do feel like this ...",Industry Discussion,NaN,74,t3_mebzea,https://www.reddit.com/r/stocks/comments/mebze...,...,0.72,[1616844550.0],"[To, explain, myself, I, really, do, feel, lik...","[What, stocks, focus, on, sexual, self, satisf...",[DD],0.03,0.843,0.127,0.9194,1
12,Submission,mdtss5,Anyone own Viacom (VIAC) here?,rockinoutwith2,"If so, my condolences...damn, this stock just ...",NaN,NaN,56,t3_mdtss5,https://www.reddit.com/r/stocks/comments/mdtss...,...,0.90,[1616779789.0],"[If, so, my, condolences, damn, this, stock, j...","[Anyone, own, Viacom, VIAC, here, ]",[VIAC],0.05,0.785,0.165,0.6129,1
13,Submission,meaofc,Suez Canal Tanker Play,TimThyTurtle,$STNG $TNK $NAT $TK $FRO\nThe Suez Canal block...,Industry Discussion,NaN,32,t3_meaofc,https://www.reddit.com/r/stocks/comments/meaof...,...,0.74,[1616838462.0],"[, STNG, TNK, NAT, TK, FRO, The, Suez, Canal, ...","[Suez, Canal, Tanker, Play]","[CEO, FRO, NAT,

In [13]:
submissions_df['sentiment_weight'] = submissions_df['Score'] * submissions_df['Upvote_Ratio']

In [14]:
submissions_df

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,...,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label,sentiment_weight
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,...,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ...",0.045,0.848,0.107,0.9937,1,66.24
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,...,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]",0.042,0.836,0.122,0.9842,1,70.47
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,...,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS],0.142,0.778,0.081,-0.7559,-1,50.02
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,...,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY...",0.031,0.871,0.098,0.9999,1,93.12
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,...,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]",0.066,0.746,0.188,0.9981,1,33.97
7,Submission,mdvvkm,Opportunity to gain about 10-14% on $MX (Bough...,RowanHarley,"Right now, $MX is trading far below what it wa...",Company Discussion,NaN,52,t3_mdvvkm,https://www.reddit.com/r/stocks/comments/mdvvk...,...,[1616785531.0],"[Right, now, MX, is, trading, far, below, what...","[Opportunity, to, gain, about, 10, 14, on, MX,...",[MX],0.085,0.763,0.152,0.8927,1,34.85
10,Submission,me0ast,Some figures summarizing what has been happeni...,futureIsYes,The Hang Seng Index contains all the Chinese g...,NaN,NaN,62,t3_me0ast,https://www.reddit.com/r/stocks/comments/me0as...,...,[1616798179.0],"[The, Hang, Seng, Index, contains, all, the, C...","[Some, figures, summarizing, what, has, been, ...",[USD],0.11,0.89,0,-0.8963,-1,28.35
11,Submission,mebzea,What stocks focus on sexual self-satisfaction?,Smur_,"To explain myself, I really do feel like this ...",Industry Discussion,NaN,74,t3_mebzea,https://www.reddit.com/r/stocks/comments/mebze...,...,[1616844550.0],"[To, explain, myself, I, really, do, feel, lik...","[What, stocks, focus, on, sexual, self, satisf...",[DD],0.03,0.843,0.127,0.9194,1,23.76
12,Submission,mdtss5,Anyone own Viacom (VIAC) here?,rockinoutwith2,"If so, my condolences...damn, this stock just ...",NaN,NaN,56,t3_mdtss5,https://www.reddit.com/r/stocks/comments/mdtss...,...,[1616779789.0],"[If, so, my, condolences, damn, this, stock, j...","[Anyone, own, Viacom, VIAC, here, ]",[VIAC],0.05,0.785,0.165,0.6129,1,24.30
13,Submission,meaofc,Suez Canal Tanker Play,TimThyTurtle,$STNG $TNK $NAT $TK $FRO\nThe Suez Canal block...,Industry Discussion,NaN,32,t3_meaofc,https://www.reddit.com/r/stocks/comments/meaof...,...,[1616838462.0],"[, STNG, TNK, NAT, TK, FRO, The, Suez, Canal, ...","[Suez, Canal, Tanker, Play]","[CEO, F

In [15]:
comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC
0,Comment,me6ezj,gsdzzz0,t3_me6ezj,Liteboyy,['So there’s 2 main differences between a warr...,NaN,NaN,False,854,[1616822889.0]
1,Comment,me6ezj,gse39f3,t3_me6ezj,Scooter-Jones,"[""Fidelity has never charged me a fee for trad...",NaN,NaN,False,119,[1616824552.0]
2,Comment,me6ezj,gsdtrfq,t3_me6ezj,WeakRhino,['Check out the subreddit r/warrantbuffet\n\nG...,NaN,NaN,False,300,[1616819237.0]
3,Comment,me6ezj,gse6uz6,t3_me6ezj,lil_layne,['Do you think traders on wall street go to th...,NaN,NaN,False,24,[1616826826.0]
4,Comment,me6ezj,gse4fxp,t3_me6ezj,bs_is_everywhere,['Oh yeah! Like that poor bastard who bought 7...,NaN,NaN,False,79,[1616825357.0]


In [16]:
comments_df['Body_Words'] = comments_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')

In [17]:
comments_df['Tickers'] = ""
for index_c, row_c in comments_df.iterrows():
    for index_s, row_s in submissions_df.iterrows():
        if row_c[1] == row_s[1]:
             comments_df.at[index_c, 'Tickers'] = row_s[14]

In [18]:
comments_df = comments_df.loc[comments_df["Tickers"] != ""]
comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers
187,Comment,me2d54,gsd1r54,t3_me2d54,pman6,['ok guys....\n\nwhat the fuck happened during...,NaN,NaN,False,35,[1616805654.0],"[, ok, guys, n, nwhat, the, fuck, happened, du...","[Here, is, a, Market, Recap, for, today, Frida..."
188,Comment,me2d54,gsdbbya,t3_me2d54,NeelAsman,"['Shout out for yesterday’s post, you nailed S...",NaN,NaN,False,5,[1616809931.0],"[, Shout, out, for, yesterday, s, post, you, n...","[Here, is, a, Market, Recap, for, today, Frida..."
189,Comment,me2d54,gsd2vy0,t3_me2d54,thatsjetfuel,"[""I'm not seeing how TSM is a bad play at this...",NaN,NaN,False,7,[1616806145.0],"[, Im, not, seeing, how, TSM, is, a, bad, play...","[Here, is, a, Market, Recap, for, today, Frida..."
190,Comment,me2d54,gsd61xo,t1_gsd1r54,hitmon_gg,['God wanted me to have a better weekend?'],NaN,NaN,False,23,[1616807556.0],"[, God, wanted, me, to, have, a, better, weeke...","[Here, is, a, Market, Recap, for, today, Frida..."
191,Comment,me2d54,gsddp75,t1_gsd1r54,bothodler,"[""I don't want to get super conspiracy-ie but ...",NaN,NaN,False,10,[1616811018.0],"[, I, dont, want, to, get, super, conspiracy, ...","[Here, is, a, Market, Recap, for, today, Frida..."


In [19]:
comments_df['negative_score'] = ""
comments_df['neutral_score'] = ""
comments_df['positive_score'] = ""
comments_df['compound_score'] = ""
#submissions_loop = submissions_df[['Title', 'Body', 'Body_Words', 'Title_Words', 'Tickers']]
for index, row in comments_df.iterrows():
    body = row[5]
    tickers = row[11]
    pol_score= sia.polarity_scores(body)
    #pol_score['submission_text'] = eval_text
    #pol_score['tickers'] = tickers
    #results.append(pol_score)
    comments_df.at[index, 'negative_score'] = pol_score['neg']
    comments_df.at[index, 'neutral_score'] = pol_score['neu']
    comments_df.at[index, 'positive_score'] = pol_score['pos']
    comments_df.at[index, 'compound_score'] = pol_score['compound']
        #submissions_loop['sentiment_scores'] = pol_score
#pprint(results[:3], width=100)

comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score
187,Comment,me2d54,gsd1r54,t3_me2d54,pman6,['ok guys....\n\nwhat the fuck happened during...,NaN,NaN,False,35,[1616805654.0],"[, ok, guys, n, nwhat, the, fuck, happened, du...","[Here, is, a, Market, Recap, for, today, Frida...",0.224,0.776,0,-0.7236
188,Comment,me2d54,gsdbbya,t3_me2d54,NeelAsman,"['Shout out for yesterday’s post, you nailed S...",NaN,NaN,False,5,[1616809931.0],"[, Shout, out, for, yesterday, s, post, you, n...","[Here, is, a, Market, Recap, for, today, Frida...",0,1,0,0
189,Comment,me2d54,gsd2vy0,t3_me2d54,thatsjetfuel,"[""I'm not seeing how TSM is a bad play at this...",NaN,NaN,False,7,[1616806145.0],"[, Im, not, seeing, how, TSM, is, a, bad, play...","[Here, is, a, Market, Recap, for, today, Frida...",0.147,0.767,0.087,-0.5423
190,Comment,me2d54,gsd61xo,t1_gsd1r54,hitmon_gg,['God wanted me to have a better weekend?'],NaN,NaN,False,23,[1616807556.0],"[, God, wanted, me, to, have, a, better, weeke...","[Here, is, a, Market, Recap, for, today, Frida...",0,0.674,0.326,0.4404
191,Comment,me2d54,gsddp75,t1_gsd1r54,bothodler,"[""I don't want to get super conspiracy-ie but ...",NaN,NaN,False,10,[1616811018.0],"[, I, dont, want, to, get, super, conspiracy, ...","[Here, is, a, Market, Recap, for, today, Frida...",0.299,0.598,0.102,-0.9497


In [20]:
comments_df['sentiment_label'] = 0
comments_df.loc[comments_df['compound_score'] > 0.2, 'sentiment_label'] = 1
comments_df.loc[comments_df['compound_score'] < -0.2, 'sentiment_label'] = -1
comments_df

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
187,Comment,me2d54,gsd1r54,t3_me2d54,pman6,['ok guys....\n\nwhat the fuck happened during...,NaN,NaN,False,35,[1616805654.0],"[, ok, guys, n, nwhat, the, fuck, happened, du...","[Here, is, a, Market, Recap, for, today, Frida...",0.224,0.776,0,-0.7236,-1
188,Comment,me2d54,gsdbbya,t3_me2d54,NeelAsman,"['Shout out for yesterday’s post, you nailed S...",NaN,NaN,False,5,[1616809931.0],"[, Shout, out, for, yesterday, s, post, you, n...","[Here, is, a, Market, Recap, for, today, Frida...",0,1,0,0,0
189,Comment,me2d54,gsd2vy0,t3_me2d54,thatsjetfuel,"[""I'm not seeing how TSM is a bad play at this...",NaN,NaN,False,7,[1616806145.0],"[, Im, not, seeing, how, TSM, is, a, bad, play...","[Here, is, a, Market, Recap, for, today, Frida...",0.147,0.767,0.087,-0.5423,-1
190,Comment,me2d54,gsd61xo,t1_gsd1r54,hitmon_gg,['God wanted me to have a better weekend?'],NaN,NaN,False,23,[1616807556.0],"[, God, wanted, me, to, have, a, better, weeke...","[Here, is, a, Market, Recap, for, today, Frida...",0,0.674,0.326,0.4404,1
191,Comment,me2d54,gsddp75,t1_gsd1r54,bothodler,"[""I don't want to get super conspiracy-ie but ...",NaN,NaN,False,10,[1616811018.0],"[, I, dont, want, to, get, super, conspiracy, ...","[Here, is, a, Market, Recap, for, today, Frida...",0.299,0.598,0.102,-0.9497,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111,Comment,me11cn,gsdhaar,t1_gsdf2zh,merlinsbeers,"[""He isn't self-made. He comes from a wealthy ...",NaN,NaN,False,2,[1616812668.0],"[, He, isnt, self, made, He, comes, from, a, w...","[Elons, pumps, lost, its, effect, Totally, uns...",0,0.846,0.154,0.6249,1
2112,Comment,me11cn,gsge5va,t1_gsdf2zh,harrison_wintergreen,['> That dude is one of the few self made bill...,NaN,NaN,False,1,[1616856792.0],"[, That, dude, is, one, of, the, few, self, ma...","[Elons, pumps, lost, its, effect, Totally, uns...",0,1,0,0,0
2113,Comment,me11cn,gsd6jo3,t1_gscuc3i,Ironleg01,"['Oh wow, news outlet reporting on thing that ...",NaN,NaN,False,4,[1616807785.0],"[, Oh, wow, news, outlet, reporting, on, thing...","[Elons, pumps, lost, its, effect, Totally, uns...",0.296,0.51,0.194,-0.25,-1
2114,Comment,me11cn,gsd0mbr,t1_gsczs3d,Tacoman404,"['Huh, Elon Musk is actually a vole?']",NaN,NaN,False,2,[1616805158.0],"[, Huh, Elon, Musk, is, actually, a, vole, ]","[Elons, pumps, lost, its, effect, Totally, uns...",0,1,0,0,0


In [21]:
sentiment_average_df = comments_df[['Submission_ID','sentiment_label']]

In [22]:
sentiment_average_df['sentiment_weight'] = comments_df['Score']
sentiment_average_df

/Users/madisonleopold/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Submission_ID,sentiment_label,sentiment_weight
187,me2d54,-1,35
188,me2d54,0,5
189,me2d54,-1,7
190,me2d54,1,23
191,me2d54,-1,10
...,...,...,...
2111,me11cn,1,2
2112,me11cn,0,1
2113,me11cn,-1,4
2114,me11cn,0,2


In [23]:
sentiment_average_df['sentiment_adjusted'] = sentiment_average_df['sentiment_label'] * sentiment_average_df['sentiment_weight']

/Users/madisonleopold/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
sentiment_average = sentiment_average_df[['Submission_ID','sentiment_adjusted']].groupby('Submission_ID').mean()

In [25]:
sentiment_average['sentiment_output'] = 0
sentiment_average.loc[sentiment_average['sentiment_adjusted'] > 0.2, 'sentiment_output'] = 1
sentiment_average.loc[sentiment_average['sentiment_adjusted'] < -0.2, 'sentiment_output'] = -1
sentiment_average.reset_index(inplace=True)
#sentiment_average = sentiment_average.set_index('Submission_ID')

In [26]:
sentiment_average

,Submission_ID,sentiment_adjusted,sentiment_output
0,mdqmi5,-2.333333,-1
1,mdr73u,0.461538,1
2,mdrafp,-0.200000,0
3,mdrf39,-0.200000,0
4,mdrs63,1.571429,1
5,mds0i6,0.500000,1
6,mds4bo,0.400000,1
7,mdsfjo,0.888889,1
8,mdssz4,0.083333,0
9,mdt1eq,1.068966,1


In [27]:
#combined_df = submissions_df.join(sentiment_average)
combined_df = pd.merge(submissions_df, sentiment_average, how='left', on='Submission_ID',
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [28]:
combined_df

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,...,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label,sentiment_weight,sentiment_adjusted,sentiment_output
0,Submission,mdqmi5,$GSX - Options activity,SaveAmerica21,$GSX - question. What is up with all the Chine...,NaN,NaN,3,t3_mdqmi5,https://www.reddit.com/r/stocks/comments/mdqmi...,...,"[, GSX, Options, activity]","[AMBO, CLEU, EEIQ, FEDU, WAFU]",0,0.957,0.043,0.34,1,3.00,-2.333333,-1.0
1,Submission,mdr73u,Short term tanker stocks for Suez Canal blocka...,Rock_it_Scientist,"Ok, so we all know that some quarter mile lon...",Industry Discussion,NaN,14,t3_mdr73u,https://www.reddit.com/r/stocks/comments/mdr73...,...,"[Short, term, tanker, stocks, for, Suez, Canal...","[CEO, DHT, DSX, FRO, NAT, SFL, TNP, TWOA]",0.08,0.869,0.051,-0.9834,-1,5.39,0.461538,1.0
2,Submission,mdrafp,XPO Logistics Inc. (XPO) Soars 2.43% on March 25,thinkB4WeSpeak,XPO Logistics Inc. (XPO) had a good day on the...,Company News,NaN,5,t3_mdrafp,https://www.reddit.com/r/stocks/comments/mdraf...,...,"[XPO, Logistics, Inc, XPO, Soars, 2, 43, on, M...","[CEO, TWOA, XPO, LTL]",0.026,0.908,0.066,0.9169,1,0.00,-0.200000,0.0
3,Submission,mdrf39,"Rookie Mistake, left with a question.",Cindermidnight2,So I misunderstood RobinHood’s day trading war...,Advice,NaN,57,t3_mdrf39,https://www.reddit.com/r/stocks/comments/mdrf3...,...,"[Rookie, Mistake, left, with, a, question, ]",[TD],0.081,0.769,0.15,0.9669,1,8.47,-0.200000,0.0
4,Submission,mdrs63,Dollar cost average,noeljvt,Is there a way to calculate dollar cost averag...,NaN,NaN,14,t3_mdrs63,https://www.reddit.com/r/stocks/comments/mdrs6...,...,"[Dollar, cost, average]",[TWOA],0,0.905,0.095,0.6046,1,0.00,1.571429,1.0
5,Submission,mds0i6,UVXY Tax situation,mkays10,"Idk If any of you are tax pros, but I’m sure p...",NaN,NaN,2,t3_mds0i6,https://www.reddit.com/r/stocks/comments/mds0i...,...,"[UVXY, Tax, situation]","[K, L, UVXY]",0.008,0.788,0.203,0.9828,1,1.50,0.500000,1.0
6,Submission,mds4bo,Anyone Else Getting Bearish on Midcaps?,JagwarRocker,"Still very early, but seeing a clear left shou...",Trades,NaN,25,t3_mds4bo,https://www.reddit.com/r/stocks/comments/mds4b...,...,"[Anyone, Else, Getting, Bearish, on, Midcaps, ]","[H, IJR, IWC, IWM, SDD]",0.059,0.901,0.04,0.1381,0,7.60,0.400000,1.0
7,Submission,mdsfjo,Can someone point out to me what i'm missing h...,Manyworldsz,Let me know if this is not the place to ask an...,NaN,NaN,9,t3_mdsfjo,https://www.reddit.com/r/stocks/comments/mdsfj...,...,"[Can, someone, point, out, to, me, what, im, m...","[AONE, MT]",0.071,0.775,0.154,0.9845,1,2.13,0.888889,1.0
8,Submission,mdssz4,Trades betting on US/China conflict?,lolkkthxbye,Tension between the US and China over Taiwan s...,Trades,NaN,12,t3_mdssz4,https://www.reddit.com/r/stocks/comments/mdssz...,...,"[Trades, betting, on, US, China, conflict, ]",[RTX],0.13,0.821,0.049,-0.8532,-1,3.90,0.083333,0.0
9,Submission,mdt1eq,[Nio vs F] Why is NIO more hyped than F?,Many_Artichoke9463,NIO 2020 revenue: ~~$16M~~ $2.4B\n\nF 2020 rev...,Company Discussion,NaN,58,t3_mdt1eq,https://www.reddit.com/r/stocks/comments/mdt1e...,...,"[, Nio, vs, F, Why, is, NIO, more, hyped, than...","[F, NIO, USD]",0.068,0.837,0.096,0.6704,1,19.50,1.068966,1.0


In [29]:
combined_df.rename(columns = {'sentiment_adjusted':'comment_sentiment'}, inplace = True)

In [30]:
combined_df['submission_sentiment'] = combined_df['sentiment_label'] * combined_df['sentiment_weight']

In [31]:
combined_df['combined_sentiment'] = combined_df['comment_sentiment'] + combined_df['submission_sentiment']

In [32]:
combined_df['final_sentiment_label'] = 0
combined_df.loc[combined_df['combined_sentiment'] > 0.2, 'final_sentiment_label'] = 1
combined_df.loc[combined_df['combined_sentiment'] < -0.2, 'final_sentiment_label'] = -1
#combined_df.reset_index(inplace=True)

In [33]:
combined_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,...,neutral_score,positive_score,compound_score,sentiment_label,sentiment_weight,comment_sentiment,sentiment_output,submission_sentiment,combined_sentiment,final_sentiment_label
0,Submission,mdqmi5,$GSX - Options activity,SaveAmerica21,$GSX - question. What is up with all the Chine...,NaN,NaN,3,t3_mdqmi5,https://www.reddit.com/r/stocks/comments/mdqmi...,...,0.957,0.043,0.34,1,3.00,-2.333333,-1.0,3.00,0.666667,1
1,Submission,mdr73u,Short term tanker stocks for Suez Canal blocka...,Rock_it_Scientist,"Ok, so we all know that some quarter mile lon...",Industry Discussion,NaN,14,t3_mdr73u,https://www.reddit.com/r/stocks/comments/mdr73...,...,0.869,0.051,-0.9834,-1,5.39,0.461538,1.0,-5.39,-4.928462,-1
2,Submission,mdrafp,XPO Logistics Inc. (XPO) Soars 2.43% on March 25,thinkB4WeSpeak,XPO Logistics Inc. (XPO) had a good day on the...,Company News,NaN,5,t3_mdrafp,https://www.reddit.com/r/stocks/comments/mdraf...,...,0.908,0.066,0.9169,1,0.00,-0.200000,0.0,0.00,-0.200000,0
3,Submission,mdrf39,"Rookie Mistake, left with a question.",Cindermidnight2,So I misunderstood RobinHood’s day trading war...,Advice,NaN,57,t3_mdrf39,https://www.reddit.com/r/stocks/comments/mdrf3...,...,0.769,0.15,0.9669,1,8.47,-0.200000,0.0,8.47,8.270000,1
4,Submission,mdrs63,Dollar cost average,noeljvt,Is there a way to calculate dollar cost averag...,NaN,NaN,14,t3_mdrs63,https://www.reddit.com/r/stocks/comments/mdrs6...,...,0.905,0.095,0.6046,1,0.00,1.571429,1.0,0.00,1.571429,1


In [44]:
ticker_sentiment = pd.DataFrame()

In [51]:
ticker_sentiment["Ticker"] = ""
ticker_sentiment["Total_Count"] = 0
ticker_sentiment["Total_Sentiment"] = 0
ticker_sentiment["Average_Sentiment"] = 0

In [52]:
index = 0
for ticker in tickers:
    ticker_sentiment.at[index, "Ticker"] = ticker.abbrev
    index += 1

In [53]:
ticker_sentiment.head()

,Ticker,Total_Count,Total_Sentiment,Average_Sentiment
0,A,0,0,0
1,AA,0,0,0
2,AAC,0,0,0
3,AACE,0,0,0
4,AACG,0,0,0


In [58]:
# Get running count of tickers mentioned (# of times mentioned) and with average sentiment
for index, row in ticker_sentiment.iterrows():
    ticker_abbrev = row[0]
    total_count = row[1]
    total_sentiment = row[2]
    for i, row in combined_df.iterrows():
        # Get column with tickers found
        #tickers_found = []
        #body_list = row[12]
        #title_list = row[13]
        ticker_set = set(row[15])
        post_sentiment = row[26]
        #body_set = set(body_list)
        #title_set = set(title_list)
        if ticker_abbrev in ticker_set:
            ticker_sentiment.at[index, "Total_Count"] += 1
            ticker_sentiment.at[index, "Total_Sentiment"] += post_sentiment
            #total_count += 1
            #total_sentiment += post_sentiment
   # ticker_sentiment.at[index, "Total_Count"] = total_count
    #ticker_sentiment.at[index, "Total_Sentiment"] = total_sentiment
        #for ticker in tickers:
           # if ticker.abbrev in ticker_set:
              #  ticker.count += 1
                #ticker.sentiment += post_sentiment
            #if ticker.abbrev == 'A' or ticker.abbrev == 'I':
               # if (ticker.name in body_set) or (ticker.name in title_set):
                    #tickers_found.append(ticker.abbrev)
           # else:
                #if (ticker.abbrev in body_set) or (ticker.abbrev in title_set):
                    #print("Abbreviation match")
                    #print(f"{index}: {ticker.abbrev}")
                    #if ticker.abbrev not in tickers_found:
                        #tickers_found.append(ticker.abbrev)
               # elif (ticker.name in body_set) or (ticker.name in title_set):
                    #print("Name match")
                    #print(f"{index}: {ticker.name}")
                    #print(f"{index}: {ticker.abbrev}")
                    #if ticker.abbrev not in tickers_found:
                       # tickers_found.append(ticker.abbrev)
        #if len(tickers_found) == 0:
            #submissions_df.at[index, "Tickers"] = ""
        #else:
            #submissions_df.at[index, "Tickers"]= tickers_found
           # print(f"{index}: {tickers_found}")

In [60]:
ticker_sentiment

,Ticker,Total_Count,Total_Sentiment,Average_Sentiment
0,A,2,2,0
1,AA,1,-1,0
2,AAC,1,0,0
3,AACE,0,0,0
4,AACG,0,0,0
...,...,...,...,...
27914,FEMY,0,0,0
27915,WKME,0,0,0
27916,LFST,0,0,0
27917,DRAY,0,0,0
